In [28]:
!pipenv install sentence_transformers

Installing sentence_transformers...
Resolving sentence_transformers...
Added sentence-transformers to Pipfile's [packages] ...
✔ Installation Succeededce_transformers...
⠋ Installing sentence_transformers...
Pipfile.lock (2cdd4f) out of date: run `pipfile lock` to update to (3fd101)...
Running $ pipenv lock then $ pipenv sync.
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking packages...
⠧ Locking packages...
Locking [dev-packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking dev-packages...
⠇ Locking dev-packages...Warning: WARNING:pipenv.patched.pip._internal.resolution.resolvelib.candidates:hyperopt 0.2.7 does not provide the extra 'all'
⠇ Locking dev-packages...
Updated Pipfile.lock (0ef11c0a6c94d8b24a58dda6513d068c4c7f29910ac6db1c5816dd74f63fd101)!
Installing dependencies from Pipfile.lock (3fd101)...
All dependencies are now up-to-date!
Installing dependencies from Pipfile.lock (3fd101

In [29]:
import pandas as pd
from elasticsearch import Elasticsearch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

/home/codespace/.local/share/virtualenvs/eat-toronto-rec-rag-pAUS2GZp/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-09-11 02:19:59--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-09-11 02:19:59 (15.0 MB/s) - ‘minsearch.py.1’ saved [3832/3832]



## Ingestion

In [3]:
df = pd.read_csv('../data/clean_data.csv')
df.columns = df.columns.str.lower()
df = df.dropna(subset=['primarytype'])

In [4]:
df.columns

Index(['id', 'nationalphonenumber', 'formattedaddress', 'rating', 'websiteuri',
       'regularopeninghours', 'displayname', 'primarytype', 'editorialsummary',
       'reviews'],
      dtype='object')

In [5]:
documents = df.to_dict(orient='records')

In [6]:
documents[0]

{'id': 'ChIJ2e-AEyMzK4gRC3Ta0xpNT2s',
 'nationalphonenumber': '(416) 322-7111',
 'formattedaddress': '2363 Yonge St, Toronto, ON M4P 2C8, Canada',
 'rating': 4.3,
 'websiteuri': 'https://www.facebook.com/marsuptown/',
 'regularopeninghours': "['Monday: 7:00\\u202fAM\\u2009–\\u20094:00\\u202fPM', 'Tuesday: 7:00\\u202fAM\\u2009–\\u20094:00\\u202fPM', 'Wednesday: 7:00\\u202fAM\\u2009–\\u20094:00\\u202fPM', 'Thursday: 7:00\\u202fAM\\u2009–\\u20099:00\\u202fPM', 'Friday: 7:00\\u202fAM\\u2009–\\u20099:00\\u202fPM', 'Saturday: 7:00\\u202fAM\\u2009–\\u20099:00\\u202fPM', 'Sunday: 7:00\\u202fAM\\u2009–\\u20094:00\\u202fPM']",
 'displayname': 'Mars Diner',
 'primarytype': 'american_restaurant',
 'editorialsummary': 'Straightforward eatery with an old-school vibe for diner-style fare from eggs to burgers to pie.',
 'reviews': 'This restaurant is a beloved family spot that offers a delightful retro experience reminiscent of 60s-style American diners. Known for its fantastic value, the meals are ge

In [45]:
es_client = Elasticsearch('http://localhost:9200')

In [8]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "id": {"type": "text"},
            "nationalphonenumber": {"type": "text"},
            "formattedaddress": {"type": "text"},
            "rating": {"type": "float"},
            "websiteuri": {"type": "text"},
            "regularopeninghours": {"type": "text"},
            "displayname": {"type": "text"},
            "primarytype": {"type": "text"},
            "editorialsummary": {"type": "text"},
            "reviews": {"type": "text"}
        }
    }
}

index_name = "restaurants-reviews"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'restaurants-reviews'})

In [9]:
for doc in tqdm(documents):
    if pd.isna(doc.get('nationalphonenumber')):
        doc['nationalphonenumber'] = " "  
    if pd.isna(doc.get('editorialsummary')):
        doc['editorialsummary'] = " "  
    if pd.isna(doc.get('websiteuri')):
        doc['websiteuri'] = " "  
    if pd.isna(doc.get('regularopeninghours')):
        doc['regularopeninghours'] = " "  
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 2025/2025 [00:33<00:00, 60.43it/s]


In [10]:
query = "I would like to try something spicy boneless"

In [11]:
def elastic_search(query):

    search_query = {
        "size": 10,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["review^3", "editorialsummary", "primarytype^2"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [12]:
elastic_search(query)

[{'id': 'ChIJmR2QYSDN1IkRXhOvOgdTBNk',
  'nationalphonenumber': '(647) 347-5570',
  'formattedaddress': '181 Eglinton Ave E, Toronto, ON M4P 1J4, Canada',
  'rating': 3.7,
  'websiteuri': 'https://www.popeyeschicken.ca/',
  'regularopeninghours': "['Monday: 10:30\\u202fAM\\u2009–\\u20092:00\\u202fAM', 'Tuesday: 10:30\\u202fAM\\u2009–\\u20093:00\\u202fAM', 'Wednesday: 10:30\\u202fAM\\u2009–\\u20092:00\\u202fAM', 'Thursday: 10:30\\u202fAM\\u2009–\\u20093:00\\u202fAM', 'Friday: 10:30\\u202fAM\\u2009–\\u20093:00\\u202fAM', 'Saturday: 10:30\\u202fAM\\u2009–\\u20093:00\\u202fAM', 'Sunday: 10:30\\u202fAM\\u2009–\\u20092:00\\u202fAM']",
  'displayname': 'Popeyes Louisiana Kitchen',
  'primarytype': 'restaurant',
  'editorialsummary': 'Counter-serve chain offering Louisiana comfort food from spicy fried chicken to red beans & rice.',
  'reviews': "At this Popeyes location, customers typically enjoy fresh and flavorful meals, especially the delicious chicken, wings, and family meals. However, re

In [6]:
import minsearch

In [7]:
index = minsearch.Index(
    text_fields=['primarytype','reviews'],
    keyword_fields=['id']
)

In [8]:
index.fit(documents)

## RAG flow

In [13]:
from openai import OpenAI

client = OpenAI()

In [10]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [14]:
prompt_template = """
You're an expert in Toronto restaurants. Make three recommendations to the QUERY based on the CONTEXT from our restaurant database.
Use only the facts from the CONTEXT when making a recommendation. If possible mention the direction, displayname, and phone of the restaurant.
Do not use bold or italics.

QUERY: {question}

CONTEXT:
{context}
""".strip()

entry_template = """
nationalphonenumber: {nationalphonenumber}
formattedaddress: {formattedaddress}
rating: {rating}
displayname: {displayname}
primarytype: {primarytype}
editorialsummary: {editorialsummary}
reviews: {reviews}
""".strip()

def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [15]:
def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [16]:
def rag(query, model='gpt-4o-mini'):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    #print(prompt)
    answer = llm(prompt, model=model)
    return answer

In [18]:
query = 'I would like to try something spicy boneless'
answer = rag(query)
print(answer)

Here are three recommendations for spicy boneless options in Toronto:

1. Popeyes Louisiana Kitchen  
   - Direction: 181 Eglinton Ave E, Toronto, ON M4P 1J4, Canada  
   - Phone: (647) 347-5570  
   - Summary: This counter-serve chain offers Louisiana comfort food, including spicy fried chicken and wings. While there have been some concerns about quality and service consistency, many patrons enjoy the flavorful meals.

2. Tokyo Hot Fried Chicken  
   - Direction: 656 College St, Toronto, ON M6G 1B8, Canada  
   - Phone: (647) 901-8432  
   - Summary: Known for its crispy and juicy fried chicken sandwiches, this spot features Japanese-inspired flavors. Many customers praise the freshness and unique taste of the offerings, especially with its flavorful sauces.

3. Kobi Korean BBQ  
   - Direction: 100 Steeles Ave W, Thornhill, ON L4J 7Y1, Canada  
   - Phone: (905) 597-1548  
   - Summary: This vibrant Korean BBQ restaurant specializes in diverse dishes, including a signature spicy chic

## Retrieval Evaluation Elastic Search

In [19]:
df_question = pd.read_csv('../data/ground-truth-retrieval.csv')

In [20]:
df_question.head()

,id,queries
0,ChIJ2e-AEyMzK4gRC3Ta0xpNT2s,looking for a retro diner experience with brea...
1,ChIJ2e-AEyMzK4gRC3Ta0xpNT2s,want a cozy place for brunch with generous por...
2,ChIJ2e-AEyMzK4gRC3Ta0xpNT2s,craving light and fluffy pancakes in a nostalg...
3,ChIJ2e-AEyMzK4gRC3Ta0xpNT2s,interested in excellent service and welcoming ...
4,ChIJ2e-AEyMzK4gRC3Ta0xpNT2s,seeking savory dishes like eggs benedict and s...


In [21]:
ground_truth = df_question.to_dict(orient='records')

In [22]:
ground_truth[0]

{'id': 'ChIJ2e-AEyMzK4gRC3Ta0xpNT2s',
 'queries': 'looking for a retro diner experience with breakfast options'}

In [25]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [20]:
def minsearch_search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [24]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [22]:
from tqdm.auto import tqdm

In [26]:
evaluate(ground_truth, lambda q: elastic_search(q['queries']))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 8205/8205 [00:18<00:00, 452.19it/s]


{'hit_rate': 0.1445460085313833, 'mrr': 0.1051001615352622}

## Vector search

In [30]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/home/codespace/.local/share/virtualenvs/eat-toronto-rec-rag-pAUS2GZp/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [31]:
v = model.encode('I would like to try something spicy boneless')

In [32]:
len(v)

384

In [34]:
es_client = Elasticsearch('http://localhost:9200')

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "id": {"type": "text"},
            "nationalphonenumber": {"type": "text"},
            "formattedaddress": {"type": "text"},
            "rating": {"type": "float"},
            "websiteuri": {"type": "text"},
            "regularopeninghours": {"type": "text"},
            "displayname": {"type": "text"},
            "primarytype": {"type": "text"},
            "editorialsummary": {"type": "text"},
            "reviews": {"type": "text"},
            "reviews_vector":{
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "primarytype_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            
        }
    }
}

index_name = "restaurants-reviews"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'restaurants-reviews'})

In [36]:
for doc in tqdm(documents):
    review = doc['reviews']
    primarytype = doc['primarytype']

    doc['reviews_vector'] = model.encode(review)
    doc['primarytype_vector'] = model.encode(primarytype)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 2025/2025 [01:40<00:00, 20.20it/s]


In [37]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 2025/2025 [00:36<00:00, 55.59it/s]


In [38]:
query = 'I would like to try something spicy boneless'

In [39]:
v_q = model.encode(query)

In [ ]:
model.

In [51]:
def elastic_search_knn(field, vector):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 10,
        "num_candidates": 10000,
    }

    search_query = {
        "knn": knn,
        "_source": ["id", "nationalphonenumber", "formattedaddress", "rating", "websiteuri", "regularopeninghours", "displayname", "primarytype", "editorialsummary", "reviews"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [52]:
def reviews_vector_knn(q):
    review = q['queries']

    v_q = model.encode(review)

    return elastic_search_knn('reviews_vector', v_q)

In [53]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [54]:
evaluate(ground_truth, reviews_vector_knn)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 8205/8205 [02:51<00:00, 47.98it/s]


{'hit_rate': 0.33418647166361976, 'mrr': 0.20513290386233649}

In [50]:
elastic_search_knn('reviews_vector', v_q)

[{'regularopeninghours': "['Monday: 11:30\\u202fAM\\u2009–\\u200910:30\\u202fPM', 'Tuesday: 11:30\\u202fAM\\u2009–\\u200910:30\\u202fPM', 'Wednesday: 11:30\\u202fAM\\u2009–\\u200910:30\\u202fPM', 'Thursday: 11:30\\u202fAM\\u2009–\\u200910:30\\u202fPM', 'Friday: 11:30\\u202fAM\\u2009–\\u200910:30\\u202fPM', 'Saturday: 11:30\\u202fAM\\u2009–\\u200910:30\\u202fPM', 'Sunday: 11:30\\u202fAM\\u2009–\\u200910:30\\u202fPM']",
  'websiteuri': 'https://www.kintonramen.com/location/kinton-ramen-yonge-eglinton/?utm_source=google&utm_medium=organic&utm_campaign=gbp-listing&utm_content=kinton-ramen-yonge-eglinton',
  'primarytype': 'ramen_restaurant',
  'reviews': 'This restaurant offers a vibrant dining experience, particularly noted during dinner with friends, highlighted by their tasty veg gyoza served with a tangy sauce. The main attraction is their ramen, featuring options like Spicy Beef, Spicy Chicken, and Spicy Veg; many reviewers prefer the Spicy Beef and Veg for their robust flavors, while

## Finding the best parameters

In [24]:
df_validation = df_question[:1000]
df_test = df_question[1000:]

In [25]:
import random

def simple_optimize(param_ranges, objective_function, n_iterations=10):
    best_params = None
    best_score = float('-inf')  # Assuming we're minimizing. Use float('-inf') if maximizing.

    for _ in range(n_iterations):
        # Generate random parameters
        current_params = {}
        for param, (min_val, max_val) in param_ranges.items():
            if isinstance(min_val, int) and isinstance(max_val, int):
                current_params[param] = random.randint(min_val, max_val)
            else:
                current_params[param] = random.uniform(min_val, max_val)
        
        # Evaluate the objective function
        current_score = objective_function(current_params)
        
        # Update best if current is better
        if current_score > best_score:  # Change to > if maximizing
            best_score = current_score
            best_params = current_params
    
    return best_params, best_score

In [26]:
gt_val = df_validation.to_dict(orient='records')

In [27]:
def minsearch_search(query, boost=None):
    if boost is None:
        boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [28]:
documents[0]

{'id': 'ChIJ2e-AEyMzK4gRC3Ta0xpNT2s',
 'nationalphonenumber': '(416) 322-7111',
 'formattedaddress': '2363 Yonge St, Toronto, ON M4P 2C8, Canada',
 'rating': 4.3,
 'websiteuri': 'https://www.facebook.com/marsuptown/',
 'regularopeninghours': "['Monday: 7:00\\u202fAM\\u2009–\\u20094:00\\u202fPM', 'Tuesday: 7:00\\u202fAM\\u2009–\\u20094:00\\u202fPM', 'Wednesday: 7:00\\u202fAM\\u2009–\\u20094:00\\u202fPM', 'Thursday: 7:00\\u202fAM\\u2009–\\u20099:00\\u202fPM', 'Friday: 7:00\\u202fAM\\u2009–\\u20099:00\\u202fPM', 'Saturday: 7:00\\u202fAM\\u2009–\\u20099:00\\u202fPM', 'Sunday: 7:00\\u202fAM\\u2009–\\u20094:00\\u202fPM']",
 'displayname': 'Mars Diner',
 'primarytype': 'american_restaurant',
 'editorialsummary': 'Straightforward eatery with an old-school vibe for diner-style fare from eggs to burgers to pie.',
 'reviews': 'This restaurant is a beloved family spot that offers a delightful retro experience reminiscent of 60s-style American diners. Known for its fantastic value, the meals are ge

In [29]:
param_ranges = {
    'displayname': (0.0, 3.0),
    'primarytype': (0.0, 3.0),
    'editorialsummary': (0.0, 3.0),
    'reviews': (0.0, 3.0),
}

def objective(boost_params):
    def search_function(q):
        return minsearch_search(q['queries'], boost_params)

    results = evaluate(gt_val, search_function)
    return results['mrr']

In [30]:
simple_optimize(param_ranges, objective, n_iterations=20)

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

({'displayname': 0.9280361750983652,
  'primarytype': 0.6526625675538263,
  'editorialsummary': 0.1523607227964593,
  'reviews': 2.404307768018854},
 0.5637051587301589)

In [32]:
def minsearch_improved(query):
    boost = {
        'displayname': 1.07,
        'primarytype': 0.14,
        'editorialsummary': 1.25,
        'reviews': 2.79
    }

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

evaluate(ground_truth, lambda q: minsearch_improved(q['queries']))

  0%|          | 0/8205 [00:00<?, ?it/s]

{'hit_rate': 0.7074954296160878, 'mrr': 0.5425317508681313}

## RAG evaluation

In [48]:
prompt2_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given query.
It is very important that you consider that the request for the dish, atmosphere, 
type of meal, among others in the query is included in the response.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {query}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the query
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [55]:
len(ground_truth)

8205

In [56]:
record = ground_truth[0]

In [54]:
prompt = prompt2_template.format(query=query, answer_llm=answer)
print(prompt)

You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given query.
It is very important that you consider that the request for the dish, atmosphere, 
type of meal, among others in the query is included in the response.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: I want a nice pleace where I can eat the spicy chicken wings
Generated Answer: Here are three recommendations for enjoying spicy chicken wings in Toronto:

1. Wow! Wing House  
   Located at 772 College St, Toronto, ON M6G 1C6, Canada, this restaurant has a rating of 4.2. They offer an extensive selection of over 100 wing flavors, including the popular "Devils Ranch Wings." The wings are known for their crispy exterior and juicy interior. You can reach them at (647) 547-5779.

2. The Wing Shop  
   Situated at 211 Queen St E, Toronto, ON M5A 1S2, 

In [58]:
import json

In [59]:
df_sample = df_question.sample(n=200, random_state=1)

In [60]:
sample = df_sample.to_dict(orient='records')

In [65]:
evaluations = []

for record in tqdm(sample):
    question = record['queries']
    answer_llm = rag(question) 

    prompt = prompt2_template.format(
        query=query,
        answer_llm=answer
    )

    evaluation = llm(prompt)
    evaluation = json.loads(evaluation)

    evaluations.append((record, answer_llm, evaluation))

  0%|          | 0/200 [00:00<?, ?it/s]

In [75]:
df_eval = pd.DataFrame(evaluations, columns=['record', 'answer', 'evaluation'])

df_eval['id'] = df_eval.record.apply(lambda d: d['id'])
df_eval['queries'] = df_eval.record.apply(lambda d: d['queries'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])

del df_eval['record']
del df_eval['evaluation']

In [76]:
df_eval.relevance.value_counts(normalize=True)

relevance
RELEVANT    1.0
Name: proportion, dtype: float64

In [70]:
df_eval.to_csv('../data/rag-eval-gpt-4o-mini.csv', index=False)

In [73]:
print(df_eval['answer'][4])

Based on your request for a busy ice cream spot in downtown Toronto that accommodates different dietary needs and does not accept credit cards, I recommend the following:

1. Hollywood Gelato
   - Direction: 1640 Bayview Ave, Toronto, ON M4G 3B7, Canada
   - Phone: (416) 544-9829
   - Summary: This cafe-style ice cream shop offers 22 housemade gelato flavors, including options for various dietary needs. It’s often busy, creating a lively atmosphere. Note that they do not accept credit cards, so be prepared to pay with cash.

2. The Fix Ice Cream Bar
   - Direction: 207 Queens Quay W, Toronto, ON M5J 1A7, Canada
   - Phone: (647) 977-2767
   - Summary: Known for its creative and seasonal soft serve flavors, The Fix offers vegan options and has received positive reviews for its friendly service. It’s a popular spot near the harbor, but they also do not accept credit cards.

3. Mizzica Gelateria & Cafe
   - Direction: 307 Queen St W, Toronto, ON M5V 2A4, Canada
   - Phone: (416) 225-2271


In [77]:
df_sample = df_question.sample(n=20, random_state=1)
sample = df_sample.to_dict(orient='records')

In [79]:
evaluations_gpt4o = []

for record in tqdm(sample):
    question = record['queries']
    answer_llm = rag(question, #model='gpt-4o') 

    prompt = prompt2_template.format(
        query=query,
        answer_llm=answer
    )

    evaluation = llm(prompt)
    evaluation = json.loads(evaluation)
    
    evaluations_gpt4o.append((record, answer_llm, evaluation))

  0%|          | 0/20 [00:00<?, ?it/s]

In [80]:
df_eval = pd.DataFrame(evaluations_gpt4o, columns=['record', 'answer', 'evaluation'])

df_eval['id'] = df_eval.record.apply(lambda d: d['id'])
df_eval['queries'] = df_eval.record.apply(lambda d: d['queries'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])

del df_eval['record']
del df_eval['evaluation']

In [81]:
df_eval.relevance.value_counts()

relevance
RELEVANT    20
Name: count, dtype: int64

In [82]:
df_eval.relevance.value_counts(normalize=True)

relevance
RELEVANT    1.0
Name: proportion, dtype: float64

In [83]:
df_eval.to_csv('../data/rag-eval-gpt-4o.csv', index=False)